In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 26.8 MB/s eta 0:00:00


In [2]:
# train_dpo.py
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model_name = "Qwen/Qwen2-0.5B-Instruct"
# model_name = "gpt2"  # swap for a small instruct model if you have GPU
model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
# training_args = DPOConfig(output_dir="Qwen2-0.5B-DPO")
# training_args = DPOConfig(output_dir="GPT2-DPO")
import wandb

wandb.init(mode='disabled')
training_args = DPOConfig(
    output_dir="Qwen2-0.5B-DPO",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=5e-5,
    max_steps=100,                     # start small; later 5k–20k
    beta=0.1,                           # DPO temperature; tune 0.05–0.3
    logging_steps=10,
    save_steps=200,
    eval_steps=200,
    bf16=True, fp16=False,              # flip depending on hardware
    remove_unused_columns=False,
    report_to=["wandb"],
)


trainer = DPOTrainer(model=model, args=training_args, processing_class=tokenizer, train_dataset=train_dataset)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


Extracting prompt in train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/62135 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.488700
20,2.312700
30,2.129200
40,2.243700
50,1.979800
60,2.286900
70,2.018900
80,2.356100
90,1.877300
100,1.827000


TrainOutput(global_step=100, training_loss=2.1520158767700197, metrics={'train_runtime': 564.0713, 'train_samples_per_second': 2.837, 'train_steps_per_second': 0.177, 'total_flos': 0.0, 'train_loss': 2.1520158767700197, 'epoch': 0.025749967812540235})

In [6]:
import transformers
print(transformers.__version__)

4.57.1


In [8]:
from transformers import pipeline, infer_device

device = infer_device()

pipeline = pipeline("text-generation", model=model, device=device, tokenizer=tokenizer)

Device set to use cuda


In [13]:
import torch
# Example for text generation
prompt = "This movie was really"
inputs = tokenizer.encode(prompt, return_tensors="pt")

# (Optional) Move inputs to GPU if the model is on GPU
if torch.cuda.is_available():
    inputs = inputs.to("cuda")

outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


This movie was really a,,,,,,,,,,,,, and, ,,,,,,,,,,,,,,,,,,,,, or,,,,,,,,


In [14]:
pipeline("The secret to baking a good cake is ")

[{'generated_text': 'The secret to baking a good cake is 1,,,,, and,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, or,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, in the,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,'}]